# InSAR Processing with SARPYX

This notebook demonstrates Interferometric Synthetic Aperture Radar (InSAR) processing using SARPYX.

## Overview

InSAR is a powerful technique that uses the phase difference between two SAR images to measure:
- Ground deformation
- Digital elevation models
- Surface changes over time

SARPYX provides comprehensive InSAR processing capabilities including:
- Interferogram generation
- Phase unwrapping
- Coherence estimation
- Atmospheric correction
- Time series analysis

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import datetime as dt

# SARPYX InSAR modules
from sarpyx.science import InSARProcessor
from sarpyx.science.insar import (
    CoregistrationProcessor,
    InterferogramGenerator,
    PhaseUnwrapper,
    CoherenceEstimator
)
from sarpyx.utils import setup_logging

# Set up logging
setup_logging(level='INFO')

print("✓ SARPYX InSAR modules imported successfully")

## InSAR Data Preparation

First, let's prepare our SAR data for interferometric processing:

In [ ]:
# Define paths to SAR image pairs
data_dir = Path("../../data")
master_image = data_dir / "S1A_IW_SLC__1SDV_20231201T060000_20231201T060030_051234_062B5E_1234.SAFE"
slave_image = data_dir / "S1A_IW_SLC__1SDV_20231213T060000_20231213T060030_051236_062B60_5678.SAFE"

# Create output directory
output_dir = Path("outputs/insar_processing")
output_dir.mkdir(parents=True, exist_ok=True)

# Initialize InSAR processor
insar_processor = InSARProcessor()

# Check if data exists
if master_image.exists() and slave_image.exists():
    print(f"✓ Master image: {master_image.name}")
    print(f"✓ Slave image: {slave_image.name}")
    data_available = True
else:
    print("⚠ Sample data not found. Using simulated processing.")
    data_available = False

# Define processing parameters
insar_config = {
    'polarization': 'VV',
    'subswath': 'IW2',  # Focus on middle subswath
    'burst_selection': [2, 3, 4],  # Select specific bursts
    'azimuth_looks': 4,
    'range_looks': 1,
    'coherence_threshold': 0.3
}

print(f"\nInSAR Configuration:")
for key, value in insar_config.items():
    print(f"  {key}: {value}")

## Step 1: Image Coregistration

Precise coregistration is crucial for InSAR processing:

In [ ]:
# Initialize coregistration processor
coreg_processor = CoregistrationProcessor()

# Configure coregistration parameters
coreg_params = {
    'coregistration_method': 'DEM_assisted',
    'dem_name': 'SRTM 1Sec HGT',
    'interpolation_method': 'bicubic',
    'cross_correlation_threshold': 0.1,
    'max_iterations': 10
}

if data_available:
    print("Performing image coregistration...")
    
    # Perform coregistration
    coregistered_stack = coreg_processor.coregister_images(
        master=master_image,
        slave=slave_image,
        output_dir=output_dir,
        **coreg_params
    )
    
    print(f"✓ Coregistration completed")
    print(f"Output: {coregistered_stack}")
    
else:
    print("Simulating image coregistration...")
    print("[INFO] Loading master image...")
    print("[INFO] Loading slave image...")
    print("[INFO] Computing initial coregistration using orbit information...")
    print("[INFO] Refining coregistration using DEM assistance...")
    print("[INFO] Cross-correlation threshold: 0.1")
    print("[INFO] Applying geometric corrections...")
    print("[INFO] Resampling slave image to master geometry...")
    print("✓ Coregistration simulation completed")

# Display coregistration quality metrics (simulated)
print("\nCoregistration Quality Metrics:")
print(f"  Average cross-correlation: 0.85")
print(f"  RMS coregistration error: 0.12 pixels")
print(f"  Maximum offset: 0.8 pixels")
print(f"  Coverage: 98.5%")

## Step 2: Interferogram Generation

Generate interferograms from the coregistered image pair:

In [ ]:
# Initialize interferogram generator
ifg_generator = InterferogramGenerator()

# Configure interferogram parameters
ifg_params = {
    'include_flat_earth_phase': False,
    'topographic_phase_removal': True,
    'dem_name': 'SRTM 1Sec HGT',
    'azimuth_looks': insar_config['azimuth_looks'],
    'range_looks': insar_config['range_looks']
}

if data_available:
    print("Generating interferogram...")
    
    # Generate interferogram
    interferogram = ifg_generator.generate_interferogram(
        coregistered_stack,
        output_dir / "interferogram.dim",
        **ifg_params
    )
    
    print(f"✓ Interferogram generated: {interferogram}")
    
else:
    print("Simulating interferogram generation...")
    print("[INFO] Computing complex interferogram...")
    print("[INFO] Removing flat earth phase...")
    print("[INFO] Removing topographic phase using SRTM DEM...")
    print("[INFO] Applying multilooking (4x1)...")
    print("[INFO] Computing interferometric phase...")
    print("✓ Interferogram generation simulation completed")

# Simulate interferogram statistics
print("\nInterferogram Statistics:")
print(f"  Temporal baseline: 12 days")
print(f"  Perpendicular baseline: 45.2 meters")
print(f"  Scene dimensions: 6250 x 4000 pixels (after multilooking)")
print(f"  Pixel spacing: 40m x 40m")
print(f"  Phase standard deviation: 1.2 radians")

## Step 3: Coherence Estimation

Estimate the interferometric coherence to assess data quality:

In [ ]:
# Initialize coherence estimator
coherence_estimator = CoherenceEstimator()

# Configure coherence parameters
coherence_params = {
    'window_size': [5, 5],  # 5x5 estimation window
    'coherence_method': 'complex',
    'bias_correction': True
}

# Generate simulated coherence data for visualization
def generate_coherence_map(width=200, height=150):
    """Generate a realistic coherence map for demonstration"""
    
    # Create base coherence pattern
    x = np.linspace(0, 10, width)
    y = np.linspace(0, 7.5, height)
    X, Y = np.meshgrid(x, y)
    
    # High coherence in urban areas (simulated)
    urban_areas = ((X-3)**2 + (Y-2)**2 < 1) | ((X-7)**2 + (Y-5)**2 < 0.8)
    
    # Water bodies (low coherence)
    water = ((X-1.5)**2 + (Y-4)**2 < 0.5) | ((X-8.5)**2 + (Y-1.5)**2 < 0.3)
    
    # Vegetation (medium coherence with temporal decorrelation)
    vegetation = ((X-5)**2 + (Y-3)**2 < 2) & ~urban_areas & ~water
    
    # Base coherence
    coherence = 0.4 + 0.3 * np.random.random((height, width))
    
    # Apply land cover effects
    coherence[urban_areas] = 0.7 + 0.25 * np.random.random(np.sum(urban_areas))
    coherence[water] = 0.05 + 0.15 * np.random.random(np.sum(water))
    coherence[vegetation] = 0.25 + 0.3 * np.random.random(np.sum(vegetation))
    
    return np.clip(coherence, 0, 1)

# Generate coherence map
coherence_map = generate_coherence_map()

# Visualize coherence
plt.figure(figsize=(12, 8))
plt.imshow(coherence_map, cmap='viridis', vmin=0, vmax=1, extent=[0, 10, 0, 7.5])
plt.colorbar(label='Interferometric Coherence')
plt.title('Interferometric Coherence Map')
plt.xlabel('Range (km)')
plt.ylabel('Azimuth (km)')

# Add annotations for different land cover types
plt.text(3, 2, 'Urban\n(High coherence)', ha='center', va='center', 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
plt.text(1.5, 4, 'Water\n(Low coherence)', ha='center', va='center',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
plt.text(5, 3, 'Vegetation\n(Medium coherence)', ha='center', va='center',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Coherence statistics
print("Coherence Statistics:")
print(f"  Mean coherence: {np.mean(coherence_map):.3f}")
print(f"  Standard deviation: {np.std(coherence_map):.3f}")
print(f"  Pixels with coherence > 0.3: {np.sum(coherence_map > 0.3)/coherence_map.size*100:.1f}%")
print(f"  Pixels with coherence > 0.7: {np.sum(coherence_map > 0.7)/coherence_map.size*100:.1f}%")

## Step 4: Phase Unwrapping

Unwrap the interferometric phase to obtain continuous phase values:

In [ ]:
# Initialize phase unwrapper
unwrapper = PhaseUnwrapper()

# Configure unwrapping parameters
unwrap_params = {
    'method': 'snaphu',  # Statistical-cost Network-flow Algorithm
    'coherence_threshold': insar_config['coherence_threshold'],
    'cost_mode': 'SMOOTH',
    'initialization': 'MST'  # Minimum Spanning Tree
}

# Generate simulated wrapped and unwrapped phase for demonstration
def generate_phase_data(width=200, height=150):
    """Generate realistic wrapped and unwrapped phase data"""
    
    # Create a smooth deformation pattern
    x = np.linspace(0, 10, width)
    y = np.linspace(0, 7.5, height)
    X, Y = np.meshgrid(x, y)
    
    # Simulate subsidence pattern
    deformation_center_x, deformation_center_y = 6, 3
    distance = np.sqrt((X - deformation_center_x)**2 + (Y - deformation_center_y)**2)
    
    # Exponential decay subsidence
    max_subsidence = 0.05  # 5 cm
    wavelength = 0.056  # Sentinel-1 C-band wavelength in meters
    
    # Convert deformation to phase (4π * deformation / wavelength)
    unwrapped_phase = 4 * np.pi * max_subsidence * np.exp(-distance**2/2) / wavelength
    
    # Add atmospheric artifacts
    atmosphere = 0.5 * np.sin(2*np.pi*X/8) * np.cos(2*np.pi*Y/6)
    unwrapped_phase += atmosphere
    
    # Add noise
    noise_level = 0.8
    unwrapped_phase += noise_level * np.random.randn(height, width)
    
    # Create wrapped phase
    wrapped_phase = np.angle(np.exp(1j * unwrapped_phase))
    
    return wrapped_phase, unwrapped_phase

# Generate phase data
wrapped_phase, true_unwrapped_phase = generate_phase_data()

# Visualize wrapped and unwrapped phase
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Wrapped phase
im1 = ax1.imshow(wrapped_phase, cmap='hsv', vmin=-np.pi, vmax=np.pi, 
                 extent=[0, 10, 0, 7.5], aspect='auto')
ax1.set_title('Wrapped Interferometric Phase')
ax1.set_xlabel('Range (km)')
ax1.set_ylabel('Azimuth (km)')
cbar1 = plt.colorbar(im1, ax=ax1, label='Phase (radians)')
cbar1.set_ticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
cbar1.set_ticklabels(['-π', '-π/2', '0', 'π/2', 'π'])

# Unwrapped phase
im2 = ax2.imshow(true_unwrapped_phase, cmap='RdBu_r', 
                 extent=[0, 10, 0, 7.5], aspect='auto')
ax2.set_title('Unwrapped Phase')
ax2.set_xlabel('Range (km)')
ax2.set_ylabel('Azimuth (km)')
plt.colorbar(im2, ax=ax2, label='Phase (radians)')

plt.tight_layout()
plt.show()

if data_available:
    print("Performing phase unwrapping...")
    
    # Perform phase unwrapping
    unwrapped_result = unwrapper.unwrap_phase(
        interferogram,
        coherence_map,
        output_dir / "unwrapped_phase.dim",
        **unwrap_params
    )
    
    print(f"✓ Phase unwrapping completed: {unwrapped_result}")
    
else:
    print("Simulating phase unwrapping...")
    print("[INFO] Applying coherence threshold masking...")
    print("[INFO] Initializing phase unwrapping with MST...")
    print("[INFO] Running SNAPHU algorithm...")
    print("[INFO] Processing 30,000 pixels...")
    print("[INFO] Convergence achieved after 15 iterations")
    print("✓ Phase unwrapping simulation completed")

print("\nPhase Unwrapping Statistics:")
print(f"  Processed pixels: {np.sum(coherence_map > 0.3):,}")
print(f"  Phase range: {np.min(true_unwrapped_phase):.2f} to {np.max(true_unwrapped_phase):.2f} radians")
print(f"  Equivalent deformation range: {np.min(true_unwrapped_phase)*0.056/(4*np.pi)*1000:.1f} to {np.max(true_unwrapped_phase)*0.056/(4*np.pi)*1000:.1f} mm")

## Step 5: Displacement Calculation

Convert unwrapped phase to ground displacement:

In [ ]:
# Convert phase to displacement
wavelength = 0.056  # Sentinel-1 C-band wavelength in meters
displacement = true_unwrapped_phase * wavelength / (4 * np.pi)  # Line-of-sight displacement

# Convert to millimeters for better visualization
displacement_mm = displacement * 1000

# Visualize displacement map
plt.figure(figsize=(12, 8))
im = plt.imshow(displacement_mm, cmap='RdBu_r', 
                extent=[0, 10, 0, 7.5], aspect='auto')
plt.colorbar(im, label='Line-of-sight displacement (mm)', shrink=0.8)
plt.title('Ground Displacement Map\n(12-day interferogram)', fontsize=14)
plt.xlabel('Range (km)')
plt.ylabel('Azimuth (km)')

# Add contour lines
contours = plt.contour(displacement_mm, levels=10, colors='black', alpha=0.3, 
                      extent=[0, 10, 0, 7.5], linewidths=0.5)
plt.clabel(contours, inline=True, fontsize=8, fmt='%.1f mm')

# Mark the subsidence center
plt.plot(6, 3, 'k*', markersize=15, label='Deformation center')
plt.legend()

plt.tight_layout()
plt.show()

# Displacement statistics
print("Displacement Analysis:")
print(f"  Maximum subsidence: {np.min(displacement_mm):.1f} mm")
print(f"  Maximum uplift: {np.max(displacement_mm):.1f} mm")
print(f"  Mean displacement: {np.mean(displacement_mm):.1f} mm")
print(f"  Standard deviation: {np.std(displacement_mm):.1f} mm")
print(f"  Area affected by >5mm displacement: {np.sum(np.abs(displacement_mm) > 5)/displacement_mm.size*100:.1f}%")

## Step 6: Quality Assessment

Assess the quality of the InSAR results:

In [ ]:
# Quality assessment metrics
def assess_insar_quality(coherence, displacement, coherence_threshold=0.3):
    """Assess InSAR processing quality"""
    
    # Create quality mask
    quality_mask = coherence > coherence_threshold
    
    # Calculate quality metrics
    metrics = {
        'valid_pixel_percentage': np.sum(quality_mask) / quality_mask.size * 100,
        'mean_coherence': np.mean(coherence[quality_mask]),
        'displacement_std': np.std(displacement[quality_mask]),
        'phase_residuals': np.std(displacement[quality_mask]) * 4 * np.pi / wavelength
    }
    
    return metrics, quality_mask

# Assess quality
quality_metrics, quality_mask = assess_insar_quality(coherence_map, displacement_mm)

# Visualize quality assessment
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Original displacement
im1 = ax1.imshow(displacement_mm, cmap='RdBu_r', extent=[0, 10, 0, 7.5])
ax1.set_title('Original Displacement')
ax1.set_ylabel('Azimuth (km)')
plt.colorbar(im1, ax=ax1, label='Displacement (mm)')

# Quality mask
im2 = ax2.imshow(quality_mask, cmap='RdYlGn', extent=[0, 10, 0, 7.5])
ax2.set_title('Quality Mask (Coherence > 0.3)')
plt.colorbar(im2, ax=ax2, label='Valid pixels')

# Masked displacement
masked_displacement = np.where(quality_mask, displacement_mm, np.nan)
im3 = ax3.imshow(masked_displacement, cmap='RdBu_r', extent=[0, 10, 0, 7.5])
ax3.set_title('Quality-filtered Displacement')
ax3.set_xlabel('Range (km)')
ax3.set_ylabel('Azimuth (km)')
plt.colorbar(im3, ax=ax3, label='Displacement (mm)')

# Coherence vs displacement scatter plot
valid_pixels = quality_mask.flatten()
coherence_flat = coherence_map.flatten()[valid_pixels]
displacement_flat = np.abs(displacement_mm.flatten()[valid_pixels])

# Sample for plotting (to avoid overcrowding)
sample_size = min(1000, len(coherence_flat))
indices = np.random.choice(len(coherence_flat), sample_size, replace=False)

ax4.scatter(coherence_flat[indices], displacement_flat[indices], 
           alpha=0.6, s=20, c='blue')
ax4.set_xlabel('Coherence')
ax4.set_ylabel('Absolute displacement (mm)')
ax4.set_title('Coherence vs Displacement')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print quality metrics
print("InSAR Quality Assessment:")
print(f"  Valid pixel coverage: {quality_metrics['valid_pixel_percentage']:.1f}%")
print(f"  Mean coherence (valid pixels): {quality_metrics['mean_coherence']:.3f}")
print(f"  Displacement standard deviation: {quality_metrics['displacement_std']:.2f} mm")
print(f"  Phase residuals: {quality_metrics['phase_residuals']:.2f} radians")

# Quality classification
if quality_metrics['valid_pixel_percentage'] > 70 and quality_metrics['mean_coherence'] > 0.6:
    quality_class = "Excellent"
elif quality_metrics['valid_pixel_percentage'] > 50 and quality_metrics['mean_coherence'] > 0.4:
    quality_class = "Good"
elif quality_metrics['valid_pixel_percentage'] > 30 and quality_metrics['mean_coherence'] > 0.3:
    quality_class = "Fair"
else:
    quality_class = "Poor"
    
print(f"\nOverall InSAR quality: {quality_class}")

## Advanced Analysis: Time Series Profile

Extract displacement time series for specific points of interest:

In [ ]:
# Define points of interest
poi_coords = [
    (6.0, 3.0, "Subsidence center"),
    (3.0, 2.0, "Urban area"),
    (8.5, 5.5, "Reference point"),
    (1.5, 4.0, "Water body")
]

# Simulate time series data (in practice, this would come from multiple interferograms)
dates = [dt.date(2023, 11, 1) + dt.timedelta(days=12*i) for i in range(10)]
date_labels = [d.strftime('%Y-%m-%d') for d in dates]

# Generate realistic time series
def generate_time_series(x, y, poi_name, dates):
    """Generate realistic displacement time series for a point"""
    
    n_dates = len(dates)
    
    if "Subsidence center" in poi_name:
        # Accelerating subsidence
        trend = -2.0 * np.arange(n_dates)**1.2
        seasonal = 3.0 * np.sin(2*np.pi*np.arange(n_dates)/12)
        noise = 2.0 * np.random.randn(n_dates)
    elif "Urban" in poi_name:
        # Stable with some seasonal variation
        trend = -0.5 * np.arange(n_dates)
        seasonal = 1.5 * np.sin(2*np.pi*np.arange(n_dates)/12)
        noise = 1.5 * np.random.randn(n_dates)
    elif "Reference" in poi_name:
        # Stable reference point
        trend = 0.1 * np.arange(n_dates)
        seasonal = 0.5 * np.sin(2*np.pi*np.arange(n_dates)/12)
        noise = 1.0 * np.random.randn(n_dates)
    else:  # Water body
        # High noise, unreliable measurements
        trend = np.zeros(n_dates)
        seasonal = np.zeros(n_dates)
        noise = 8.0 * np.random.randn(n_dates)
    
    return trend + seasonal + noise

# Generate time series for all POIs
plt.figure(figsize=(14, 8))

colors = ['red', 'blue', 'green', 'orange']
markers = ['o', 's', '^', 'D']

for i, (x, y, name) in enumerate(poi_coords):
    displacement_ts = generate_time_series(x, y, name, dates)
    
    plt.plot(dates, displacement_ts, 
             color=colors[i], marker=markers[i], 
             linewidth=2, markersize=6, 
             label=f"{name} ({x:.1f}, {y:.1f})")

plt.xlabel('Date')
plt.ylabel('Cumulative displacement (mm)')
plt.title('InSAR Time Series Analysis\nDisplacement at Points of Interest')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Add zero reference line
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Print time series statistics
print("Time Series Analysis Summary:")
for i, (x, y, name) in enumerate(poi_coords):
    displacement_ts = generate_time_series(x, y, name, dates)
    velocity = (displacement_ts[-1] - displacement_ts[0]) / (len(dates) - 1) * 365/12  # mm/year
    print(f"  {name}: {velocity:.1f} mm/year")

## Summary and Best Practices

This notebook demonstrated a complete InSAR processing workflow using SARPYX:

### Processing Steps Covered:
1. **Image Coregistration**: Precise alignment of SAR image pairs
2. **Interferogram Generation**: Computing phase differences
3. **Coherence Estimation**: Assessing measurement reliability
4. **Phase Unwrapping**: Converting wrapped phase to continuous values
5. **Displacement Calculation**: Converting phase to ground movement
6. **Quality Assessment**: Evaluating result reliability

### Key InSAR Considerations:
- **Temporal Baseline**: Shorter intervals reduce decorrelation
- **Perpendicular Baseline**: Affects height ambiguity and coherence
- **Coherence Threshold**: Balance between coverage and quality
- **Atmospheric Effects**: Consider correction for large-scale artifacts
- **Reference Point**: Choose stable areas for displacement reference

### Best Practices:
1. Always validate coherence before interpreting displacement
2. Use multiple interferograms for time series analysis
3. Consider atmospheric correction for precise measurements
4. Validate results with independent data (GPS, leveling)
5. Account for line-of-sight geometry in interpretation

## Next Steps

- Explore **Time Series Analysis** in `04_time_series_analysis.ipynb`
- Learn **Advanced Workflows** in `05_advanced_workflows.ipynb`
- Check out **Multi-temporal InSAR** techniques

---

*For more information on InSAR theory and applications, see the SARPYX documentation and scientific literature.*